# <center> Twitter Sentiment Analysis_Kaggle Dataset </center>

# Importing Necessary Libraries

In [4]:
import sklearn
import numpy as np
import pandas as pd

# Importing Data

In [55]:
# training data
train = pd.read_csv("train.csv")

# test data
test = pd.read_csv("test.csv")

# Data Exploration (EDA)

In [56]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [57]:
test.tail()

,id,tweet
17192,49155,thought factory: left-right polarisation! #tru...
17193,49156,feeling like a mermaid ð #hairflip #neverre...
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...
17195,49158,"happy, at work conference: right mindset leads..."
17196,49159,"my song ""so glad"" free download! #shoegaze ..."


In [58]:
# non-racist/sexist tweets
sum(train["label"] == 0)

29720

In [59]:
# racist/sexist tweets
sum(train["label"] == 1)

2242

In [60]:
# check if there are any missing/null values
train.isnull().sum()
#train.isnull().values.any()

id       0
label    0
tweet    0
dtype: int64

# Data cleaning

In [61]:
#installing tweet-preprocessor to clean tweets
!pip install tweet-preprocessor

In [62]:
# remove special characters using the regular expression library
import re

#set up punctuations that we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [63]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def cleaned_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert tweets to lower case
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [64]:
# clean training data
train_tweets = cleaned_tweets(train["tweet"])
train_tweets = pd.DataFrame(train_tweet)

In [65]:
# appending cleaned tweets to the training data
train["cleaned_tweets"] = train_tweets

# compare the cleaned and uncleaned tweets
train.head(10)

,id,label,tweet,cleaned_tweets
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for credit i cant use cause they dont o...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,factsguide society now
5,6,0,[2/2] huge fan fare and big talking before the...,2 2 huge fan fare and big talking before they ...
6,7,0,@user camping tomorrow @user @user @user @use...,camping tomorrow danny
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams can...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land
9,10,0,@user @user welcome here ! i'm it's so #gr...,welcome here im its so


In [67]:
# clean the test data and append the cleaned tweets to the test data
test_tweet = clean_tweets(test["tweet"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
test["cleaned_tweets"] = test_tweet

# compare the cleaned and uncleaned tweets
test.head(10)

,id,tweet,cleaned_tweets
0,31963,#studiolife #aislife #requires #passion #dedic...,to find
1,31964,@user #white #supremacists want everyone to s...,want everyone to see the new and heres why
2,31965,safe ways to heal your #acne!! #altwaystohe...,safe ways to heal your
3,31966,is the hp and the cursed child book up for res...,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",rd to my amazing hilarious eli ahmir uncle dav...
5,31968,choose to be :) #momtips,choose to be
6,31969,something inside me dies ð¦ð¿â¨ eyes nes...,something inside me dies eyes ness
7,31970,#finished#tattoo#inked#ink#loveitâ¤ï¸ #â¤ï¸...,
8,31971,@user @user @user i will never understand why...,i will never understand why my dad left me whe...
9,31972,#delicious #food #lovelife #capetown mannaep...,mannaepicure


# Test and Train split


In [69]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = train.label.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(train.cleaned_tweets.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

# Vectorize tweets using CountVectorizer

CountVectorizer Example

In [71]:
from sklearn.feature_extraction.text import CountVectorizer

In [85]:
documents = ["This is one of my many Github projects based on Data science",
             "I am an aspiring Data scientist and I love working with Data!",
             "I currently work in the field of applied mathematics"]

# initializing the countvectorizer
vectorizer = CountVectorizer()

# tokenize and make the document into a matrix
document_term_matrix = vectorizer.fit_transform(documents)

# check the result
pd.DataFrame(document_term_matrix.toarray(), columns = vectorizer.get_feature_names())

,am,an,and,applied,aspiring,based,currently,data,field,github,...,on,one,projects,science,scientist,the,this,with,work,working
0,0,0,0,0,0,1,0,1,0,1,...,1,1,1,1,0,0,1,0,0,0
1,1,1,1,0,1,0,0,2,0,0,...,0,0,0,0,1,0,0,1,0,1
2,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,1,0


In [86]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

# Modeling

Apply Support Vetor machine Classifier (SVC)

In [89]:
from sklearn import svm
# classify using support machine vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

# Accuracy score using SVC


In [90]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  94.86912086766085 %
